In [ ]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms


%matplotlib inline

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.Cov = nn.Conv2d(1, 20, 5, bias=False)
        self.MaxPooling = nn.MaxPool2d(2, 2)
        self.FullyConnected = nn.Linear(12*12*20, 100, bias=False)
        self.output = nn.Linear(100, 10, bias=False)
    
    def forward(self, x):
        x = self.Cov(x)
        x = self.MaxPooling(x)
        x = x.view(-1, 12*12*20)
        x = self.FullyConnected(x)
        x = self.output(x)
        return F.log_softmax(x, dim=1)
        
    

In [ ]:
batch_size=200
learning_rate=0.01
epochs=10
log_interval=100

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=batch_size, shuffle=True)

In [ ]:
cnn = CNN()
print(cnn)

In [ ]:
optimizer = optim.SGD(cnn.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.NLLLoss()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# run the main training loop
for epoch in range(epochs):
    total_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        optimizer.zero_grad()
        cnn_out = cnn(data)
        loss = criterion(cnn_out, target)
        total_loss+=loss.data
        loss.backward()
        optimizer.step()
        if batch_idx and batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))
    print('Train Epoch: {} Avg. Training Loss: {:.6f}\n'.format(
                epoch+1,
                total_loss/len(train_loader.dataset)))

In [ ]:
test_loss = 0.0
correct = 0.0

for data, target in test_loader:
    cnn_out = cnn(data)
    # sum up batch loss
    test_loss += criterion(cnn_out, target).data
    pred = cnn_out.data.max(1)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {:.0f}/{} ({:.2%})\n'.format(
    test_loss, correct, len(test_loader.dataset),correct / len(test_loader.dataset)))

In [ ]:
torch.save(cnn, './model.pth')
cnn1 = torch.load('./model.pth')

In [ ]:
test_loss = 0.0
correct = 0.0

for data, target in test_loader:
    cnn1_out = cnn1(data)
    # sum up batch loss
    test_loss += criterion(cnn1_out, target).data
    pred = cnn1_out.data.max(1)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {:.0f}/{} ({:.2%})\n'.format(
    test_loss, correct, len(test_loader.dataset),correct / len(test_loader.dataset)))